# Nuclio - Data preperation function

## Environment

In [18]:
# nuclio: ignore
import nuclio

### Configurations

In [19]:
%%nuclio config

# Trigger
spec.triggers.retrain.kind = "cron"
spec.triggers.retrain.attributes.interval = "1h"

# Base image
spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.triggers.retrain.kind to 'cron'
%nuclio: setting spec.triggers.retrain.attributes.interval to '1h'
%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


### Commands

In [20]:
%%nuclio cmd -c

############
# installs #
############

# Utils
pip install pyarrow
pip install pandas

# Igz DB
pip install v3io_frames --upgrade

# Function
pip install dask["complete"]

### Variables

In [21]:
%%nuclio env

# DB Config
V3IO_FRAMESD=${V3IO_FRAMESD}
V3IO_USERNAME=${V3IO_USERNAME}
V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}

# Save as
SAVE_TO_TSDB=1

# Metrics
METRICS_TABLE=netops_metrics
# METRICS_TABLE=/v3io/bigdata/netops_metrics_parquet

# Features
FEATURES_TABLE=netops_features
# FEATURES_TABLE=/v3io/bigdata/netops_features_parquet


# Parallelizem
NUMBER_OF_SHARDS=4

%nuclio: setting 'V3IO_FRAMESD' environment variable
%nuclio: setting 'V3IO_USERNAME' environment variable
%nuclio: setting 'V3IO_ACCESS_KEY' environment variable
%nuclio: setting 'SAVE_TO_TSDB' environment variable
%nuclio: setting 'METRICS_TABLE' environment variable
%nuclio: setting '# METRICS_TABLE' environment variable
%nuclio: setting 'FEATURES_TABLE' environment variable
%nuclio: setting '# FEATURES_TABLE' environment variable
%nuclio: setting 'NUMBER_OF_SHARDS' environment variable


%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line


## Function

### Imports

In [22]:
# Utils
import os
import time
import pandas as pd
import itertools

# DB Connection
import v3io_frames as v3f

# Parallelization
import dask.dataframe as dd
from dask.distributed import Client

### Helper functions

In [23]:
def format_df_from_tsdb(context, df):
    df.index.names = ['timestamp', 'company', 'data_center', 'device']
    df = df.reset_index()
    df = dd.from_pandas(df, npartitions=context.shards)
    return df

In [24]:
def get_data_tsdb(context):
    df = context.v3f.read(backend='tsdb', query=f'select cpu_utilization, latency, packet_loss, throughput, is_error from {context.metrics_table}',
                          start=f'now-2h', end='now', multi_index=True)
    df = format_df_from_tsdb(context, df)
    return df

In [25]:
def get_data_parquet(context):
    # Get parquet files
    mpath = [os.path.join(context.metrics_table, file) for file in os.listdir(context.metrics_table)]
    
    # Get latest filename
    latest = max(mpath, key=os.path.getmtime)
    
    # Load parquet
    df = pd.read_parquet(latest)
    
    # To Dask
    df = format_df_from_tsdb(context, df)
    return df

In [26]:
def create_rolling_featuers(context, df, window_size: int):
    features = df.copy()
    features['key'] = features.apply(lambda row: f'{row["company"]}_{row["data_center"]}_{row["device"]}', axis=1, meta=features.compute().dtypes)
    features.set_index('key')
    features["cpu_utilization"] = features.cpu_utilization.rolling(window=window_size).mean()
    features["latency"] = features.latency.rolling(window=window_size).mean()
    features["packet_loss"] = features.packet_loss.rolling(window=window_size).mean()
    features["throughput"] = features.throughput.rolling(window=window_size).mean()
    features["is_error"] = features.is_error.rolling(window=window_size).max()
                                     
    features = features.dropna()
    features = features.drop_duplicates()

    return features

In [27]:
def set_indexes(df):
    df = df.set_index(['timestamp', 'company', 'data_center', 'device'])
    return df

In [28]:
def save_to_tsdb(context, features: pd.DataFrame):   
    context.v3f.write('tsdb', context.features_table, features)

In [29]:
def save_to_parquet(context, df: pd.DataFrame):
    print('Saving features to Parquet')
    
    # Need to fix timestamps from ns to ms if we write to parquet
    df = df.reset_index()
    df['timestamp'] = df.loc[:, 'timestamp'].astype('datetime64[ms]')
    
    # Fix indexes
    df= set_indexes(df)
    
    # Save parquet
    first_timestamp = df.index[0][0].strftime('%Y%m%dT%H%M%S')
    last_timestamp = df.index[-1][0].strftime('%Y%m%dT%H%M%S')
    filename = first_timestamp + '-' + last_timestamp + '.parquet'
    filepath = os.path.join(context.features_table, filename)
    with open(filepath, 'wb+') as f:
        df.to_parquet(f)

### Init context

In [35]:
def init_context(context):
    
    # Create Dask client
    dask_client = Client()
    setattr(context, 'dask', dask_client)  
    
    # Dask shards / CV
    setattr(context, 'shards', int(os.getenv('NUMBER_OF_SHARDS', 4)))
    
    # Get saving configuration
    is_save_to_tsdb = (int(os.getenv('SAVE_TO_TSDB', 1)) == 1)
    
    # Netops metrics table
    setattr(context, 'metrics_table', os.getenv('METRICS_TABLE', 'netops_metrics'))
    
    # Netops feautres table
    setattr(context, 'features_table', os.getenv('FEATURES_TABLE', 'netops_features'))
    
    
    # Save to TSDB
    if is_save_to_tsdb:
        # Create our DB client
        v3io_client = v3f.Client(address='framesd:8081', container='bigdata')
        setattr(context, 'v3f', v3io_client)
        
        # Create features table if neede
        context.v3f.create('tsdb', context.features_table, attrs={'rate': '1/s'}, if_exists=1)
        
        # Set TSDB reading function
        setattr(context, 'read', get_data_tsdb)
        
        # Set TSDB saving function
        setattr(context, 'write', save_to_tsdb)
        
    # Save to Parquet
    else:
         # Create saving directory if needed
        filepath = os.path.join(context.features_table)
        if not os.path.exists(filepath):
            os.makedirs(filepath)
            
        # Set Parquet reading function
        setattr(context, 'read', get_data_parquet)
        
        # Set Parquet saving function
        setattr(context, 'write', save_to_parquet)

### Handler

In [42]:
def handler(context, event):
    
    # Get data
    raw = context.read(context) 
        
    # Get minute features
    minute = create_rolling_featuers(context, raw, 3)
    column_names = {'cpu_utilization': 'cpu_utilization_minutely',
                    'latency': 'latency_minutely',
                    'packet_loss': 'packet_loss_minutely',
                    'throughput': 'throughput_minutely'}
    minute = minute.rename(columns=column_names)
    
    # Get hour features
    hour = create_rolling_featuers(context, raw, 3*60)
    column_names = {'cpu_utilization': 'cpu_utilization_hourly',
                    'latency': 'latency_hourly',
                    'packet_loss': 'packet_loss_hourly',
                    'throughput': 'throughput_hourly'}
    hour = hour.rename(columns=column_names)
    
    # Create feature vector from data sources
    features_rm = raw.merge(minute, on=['timestamp', 'company', 'data_center', 'device'], suffixes=('_raw', '_minute'))
    features_rm.compute()
    
    features = features_rm.merge(hour, on=['timestamp', 'company', 'data_center', 'device'], suffixes=('_raw', '_hourly'))
    features = features.compute()
    
    # Save feature vector to TSDB
    
    # Drop key columns
    features = features.reset_index(drop=True)
    feature_cols = [col for col in features.columns if 'key' in col]
    features = features.drop(feature_cols, axis=1)
    
    
    # Fix indexes before saving
    features = features.set_index(['timestamp', 'company', 'data_center', 'device'])
    
    # Save to TSDB
    context.write(context, features)

## Test

In [48]:
# nuclio: ignore
init_context(context)

In [49]:
# nuclio: ignore
# init_context(context)
event = nuclio.Event(body='')
output = handler(context, event)
output

## Deployment

In [45]:
%nuclio deploy -p netops -n PreProcessing -c

[nuclio.deploy] 2019-06-20 14:45:37,706 (info) Building processor image
[nuclio.deploy] 2019-06-20 14:45:43,756 (info) Pushing image
[nuclio.deploy] 2019-06-20 14:45:43,757 (info) Build complete
[nuclio.deploy] 2019-06-20 14:45:49,813 (info) Function deploy complete
[nuclio.deploy] 2019-06-20 14:45:49,818 done updating preprocessing, function address: 18.185.111.133:31293
%nuclio: function deployed
